In [23]:
#svm
import menpo.io as mio
import os
from sklearn import svm
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import math
import Models
from Models import ChangeVector, ImageData
path_to_svm_training_database = '/Programing/GR/Code/CK+/aam-images/**/**/**/*'


def process(image, crop_proportion=0.2, max_diagonal=400):
    if image.n_channels == 3:
        image = image.as_greyscale()
    image = image.crop_to_landmarks_proportion(crop_proportion)
    d = image.diagonal()
    if d > max_diagonal:
        image = image.rescale(float(max_diagonal) / d)
    return image

#process changingVector, reduce dimension from 2x68 to 1x68, by process PCA
def pca(changeVector):
    X = np.array(changeVector)
    pca_model = PCA(n_components=1)
    return pca_model.fit_transform(X)

def landmark_normalize(landmark):
    vector_max = 0
    for p in landmark:
        if(math.sqrt(float(p[0])*float(p[0]) + float(p[1])*float(p[1])) > vector_max):
            vector_max = math.sqrt(p[0]*p[0] + p[1]*p[1])
    for p in landmark:
        p[0] = p[0]/float(vector_max)
        p[1] = p[1]/float(vector_max)
    return landmark


training_images = mio.import_images(path_to_svm_training_database, verbose=True)
training_images = training_images.map(process)

path_to_facs = '/Programing/GR/Code/CK+/FACS/'
path_to_emotions = '/Programing/GR/Code/CK+/Emotion/'

#create training data

#check if example is labeled
labeled_subject = []
emotion_subject = os.listdir(path_to_emotions)
for subject in emotion_subject:
    session = os.listdir(path_to_emotions + "/" + subject)
    for s in session:
        labeled_subject.append(subject + "-" + s)

count = 0;
svm_training_data = []
while(count < len(training_images)):
    file_path = str(training_images[count].path).split("\\")
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    gt_emotion = -1
    
    #get emotion label from Emotion Folder
    emotion_checker = file_path[6] + "-" + file_path[7]
    if(emotion_checker in  labeled_subject):
        emotion_path = path_to_emotions + file_path[6] + '/' + file_path[7]
        if(len(os.listdir(emotion_path)) != 0):
            emotion_path = emotion_path + '/' + os.listdir(emotion_path)[0]
            fi = open(emotion_path)
            for line in fi:
                if(line.split()):
                    gt_emotion = int(float(line.split()[0]))
#                 print(emotion_path + ":" + str(gt_emotion))
            fi.close()
    
    #get facs from FACS folder
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    data_facs = {}
    tmp = []
    for line in fi: # read rest of lines
        if(line.split()):
            tmp.append(line.split())
    for f in tmp:
        data_facs[str(int(float(f[0])))] = int(float(f[1]))
    fi.close()
    
    landmark = []
    landmark_neutral = training_images[count].landmarks['PTS'].lms.points
    landmark_perk = training_images[count + 1].landmarks['PTS'].lms.points
    for i in range(0,68):
        landmark.append([landmark_perk[i][0] - landmark_neutral[i][0], landmark_perk[i][1] - landmark_neutral[i][1]])

#     svm_training_data.append(ChangeVector(data_facs, landmarkChange, gt_emotion))   

    svm_training_data.append(ImageData(data_facs, landmark, gt_emotion))
    count = count + 2
    

facs = []
#create facs array
for data in svm_training_data:
    for facs_code in data.facs:
        if((int(facs_code)) not in facs):
            facs.append(int(facs_code))
facs.sort()


Found 1020 assets, index the returned LazyList to import.


In [3]:
for au in svm_training_data[0].facs:
    print(svm_training_data[0].facs[au])

2
4


In [20]:
for data in svm_training_data:
    if(data.emotion == 2):
        print(svm_training_data.index(data))

559
560
561
562
563
564
565
566
567
568
569
570
575
578
581
584
587
590


In [5]:
#create model for each action unit in facs[]
models = []
au_models_score = []
for au in facs:
    x_training = []
    y_label = []
    #create label array
    for data in svm_training_data:
        if(str(au) in data.facs):
#             y_label.append(data.facs[str(au)])
            y_label.append(1)
        else:
            y_label.append(-1)
        #create training data: 1x68 array, result of PCA process
        vector = []
        for tmp in data.landmark:
            vector.append(tmp[0])
            vector.append(tmp[1])
        x_training.append(vector)
    clf = svm.SVC(kernel='linear', decision_function_shape = 'ovr')
    clf.fit(normalize(x_training, norm='max', axis=0), y_label)
    au_models_score.append(clf.score(normalize(x_training, norm='max', axis=0), y_label))
    models.append(clf)
    
#create testing data
svm_testing_data = []
path_to_svm_testing_database = "/Programing/GR/Code/CK+/test-aam-images/**/**/**/*"
testing_images = mio.import_images(path_to_svm_testing_database, verbose=True)
testing_images = testing_images.map(process)

count = 0;
while(count < len(testing_images)):
    file_path = str(testing_images[count].path).split("\\")
#     print(file_path)
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    gt_emotion = -1
    
    #get emotion label from Emotion Folder
    emotion_checker = file_path[6] + "-" + file_path[7]
    if(emotion_checker in  labeled_subject):
        emotion_path = path_to_emotions + file_path[6] + '/' + file_path[7]
        if(len(os.listdir(emotion_path)) != 0):
            emotion_path = emotion_path + '/' + os.listdir(emotion_path)[0]
            fi = open(emotion_path)
            for line in fi:
                if(line.split()):
                    gt_emotion = int(float(line.split()[0]))
#                 print(emotion_path + ":" + str(gt_emotion))
            fi.close()
    
    
    #get facs from FACS folder
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    data_facs = {}
    tmp = []
    for line in fi: # read rest of lines
        if(line.split()):
            tmp.append(line.split())
    for f in tmp:
        data_facs[str(int(float(f[0])))] = int(float(f[1]))
    fi.close()
    
#     landmarkChange = []
#     landmark_neutral = testing_images[count].landmarks['PTS'].lms.points
#     landmark_perk = testing_images[count + 1].landmarks['PTS'].lms.points
#     for i in range(0,68):
#         landmarkChange.append([landmark_perk[i][0] - landmark_neutral[i][0], landmark_perk[i][1] - landmark_neutral[i][1]])
    
#     svm_testing_data.append(ChangeVector(data_facs, landmarkChange, gt_emotion))   
    svm_testing_data.append(ImageData(data_facs, training_images[count + 1].landmarks['PTS'].lms.points, gt_emotion))
    count = count + 2

    
#evaluate trained model with test data and get score
print('#######')
print('Score: ')
au_test_score = []
for au in facs:
    x_training = []
    y_label = []
    #create label array
    for data in svm_testing_data:
        if(str(au) in data.facs):
#             y_label.append(data.facs[str(au)])
            y_label.append(1)
        else:
            y_label.append(-1)
        #create training data: 1x68 array, result of PCA process
        vector = []
        for tmp in data.landmark:
            vector.append(tmp[0])
            vector.append(tmp[1])
        x_training.append(vector)
#     print(y_label)
    au_test_score.append(models[facs.index(au)].score(normalize(x_training, norm='max', axis=0), y_label))
print(au_test_score)
print(np.mean(au_test_score))

Found 116 assets, index the returned LazyList to import.
#######
Score: 
[0.31034482758620691, 0.17241379310344829, 0.60344827586206895, 0.55172413793103448, 0.93103448275862066, 0.94827586206896552, 0.98275862068965514, 0.96551724137931039, 0.98275862068965514, 0.81034482758620685, 0.98275862068965514, 0.63793103448275867, 0.36206896551724138, 0.96551724137931039, 0.63793103448275867, 1.0, 0.25862068965517243, 1.0, 1.0, 0.81034482758620685, 0.94827586206896552, 0.37931034482758619, 0.91379310344827591, 0.89655172413793105, 1.0, 1.0, 0.98275862068965514, 1.0, 1.0, 0.91379310344827591, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
0.844827586207


In [6]:
print(au_models_score)
print(np.mean(au_models_score))

[0.94953271028037378, 0.98691588785046724, 0.89719626168224298, 0.95327102803738317, 0.9196261682242991, 0.90467289719626165, 0.99252336448598133, 0.96635514018691593, 0.95140186915887848, 0.9738317757009346, 0.9981308411214953, 0.9719626168224299, 0.95514018691588787, 0.95887850467289715, 0.9476635514018692, 0.98691588785046724, 0.9719626168224299, 0.9981308411214953, 0.99439252336448603, 0.96635514018691593, 0.95514018691588787, 0.96635514018691593, 0.91588785046728971, 0.98878504672897194, 0.9981308411214953, 0.99626168224299061, 0.9981308411214953, 0.99626168224299061, 0.9981308411214953, 0.95514018691588787, 0.9738317757009346, 0.98504672897196266, 0.9981308411214953, 0.9719626168224299, 1.0, 0.9981308411214953, 0.99626168224299061, 0.99626168224299061, 0.99252336448598133]
0.97244188833


In [17]:
print(clf)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [36]:
print(len(svm_testing_data))

29


In [56]:
#regresssion model
wrong_predict = 0
au_score = []
for data in svm_testing_data:
    print('#####')
    local_wrong_predict = 0
    local_accurate_predict = 0
    tmp = []
    predict = []
    
    for vector in data.landmark:
        tmp.append(vector[0])
        tmp.append(vector[1])
        
    for model in models:
        if(model.predict([tmp]) != -1):
#             predict.append([facs[models.index(model)],data.facs[str(facs[models.index(model)])] , model.predict([tmp])])
            predict.append([models.index(model), model.predict([tmp])[0]])
            #print(facs[models.index(model)])
            if(str(facs[models.index(model)]) not in data.facs):
                local_wrong_predict += 1
            else: 
                local_accurate_predict += 1
        else:
            if(str(facs[models.index(model)]) in data.facs):
                local_wrong_predict += 1
    print(predict)
#     print(local_accurate_predict)
    au_score.append(float(local_accurate_predict)/float(len(data.facs)))
    print("---")
    print(data.facs)
    wrong_predict += local_wrong_predict
    
# print(wrong_predict)
# print(sum(au_score)/float(len(au_score)))

#####
[[0, 1], [1, 1], [3, 1], [6, 1], [7, 1], [8, 1], [9, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '24': 2, '12': 2}
#####
[[0, 1], [1, 1], [3, 1], [8, 1], [9, 1], [10, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '12': 1}
#####
[[0, 1], [1, 1], [3, 1], [8, 1], [10, 1], [13, 1], [17, 1], [21, 1], [23, 1], [24, 1], [25, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 0}
#####
[[0, 1], [1, 1], [3, 1], [7, 1], [8, 1], [9, 1], [10, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '12': 1}
#####
[[0, 1], [1, 1], [3, 1], [10, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '17': 3}
#####
[[0, 1], [1, 1], [3, 1], [6, 1], [8, 1], [9, 1], [10, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '17':

In [43]:
#get predict result
y_label = []
for au in facs:
    x_training = []
    y_label_tmp = []
    #create label array
    for data in svm_testing_data:
        if(str(au) in data.facs):
#             if(data.facs[str(au)] == 0):
#                 y_label.append(2)
#             else:
            y_label_tmp.append(data.facs[str(au)])
#             y_label.append(1)
        else:
            y_label_tmp.append(-1)
        #create training data: 1x68 array, result of PCA process
        vector = []
        for tmp in data.landmark:
            vector.append(tmp[0])
            vector.append(tmp[1])
        x_training.append(vector)
    y_label.append(y_label_tmp)
for model in models:
    print("=====")
    print(model.predict(x_training))
    print("==")
    print(y_label[models.index(model)])

=====
[-1  2  2  2  2  2  2  2  2  2 -1 -1  2  2  2  2  2  2  2  2  2  2  2  2
  2  2 -1 -1  2]
==
[-1, -1, -1, -1, -1, -1, 2, -1, 2, 2, -1, -1, 3, -1, -1, -1, -1, -1, 2, 1, -1, 2, -1, -1, 4, 4, -1, -1, 2]
=====
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
==
[-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 1, -1, -1, -1, -1, 4, 2, -1, -1, 1]
=====
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1]
==
[-1, -1, -1, -1, -1, -1, 1, 3, 3, 4, 3, -1, 4, 4, -1, 3, 2, -1, 2, 2, -1, 2, -1, -1, 2, 3, -1, 4, 4]
=====
[2 0 0 2 2 0 0 2 2 2 2 2 2 0 2 2 2 2 0 2 2 0 2 2 0 2 2 2 2]
==
[-1, -1, -1, -1, -1, -1, -1, 1, 4, -1, 3, -1, 3, 2, -1, -1, 3, -1, 4, -1, -1, -1, 2, -1, 4, -1, -1, 1, 3]
=====
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1]
==
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1, -1]
=====
[5 5 5 5 5 5 5 5 5

In [3]:
#build rule-based system
#emotion class have name = str, criteria = function, facs required = [[]], caculate score of input
class Emotion:
    def __init__(self, name, facs_required, criteria):
        self.name = name
        self.facs_required = facs_required
        self.criteria = criteria
    
    def criteria(self, facs_input):
        return True
    
    def score(self,facs_input = []):
        if(self.criteria(facs_input) == True):
            max = 0
            for required in self.facs_required:
                au_count = 0
                for facs in facs_input:
                    if facs in required:
                        au_count += 1
                if au_count/float(len(required)) >= max:
                    max = au_count/float(len(required))
            return max
        else:
            return 0
    
def angry_criteria(facs_input):
    if(23 in facs_input):
        return True
    return False

def disgus_criteria(facs_input):
    if(9 in facs_input or 10 in facs_input):
        return True
    return False

def fear_criteria(facs_input):
    if(1 in facs_input and 2 in facs_input and 3 in facs_input):
        return True
    return False

def surprise_criteria(facs_input):
    if(1 in facs_input and 2 in facs_input):
        return True
    if(5 in facs_input):
        return True
    return False

def sadness_criteria(facs_input):
    return True

def happy_criteria(facs_input):
    if(12 in facs_input):
        return True
    return False

def contempt_criteria(facs_input):
    if(14 in facs_input):
        return True
    return False

happy = Emotion('happy', [[6,12]], happy_criteria)
sadness = Emotion('sadness', [[1,4,5], [6,15], [1,4,15]], sadness_criteria)
surprise = Emotion('surprise', [[1,2,5,26]], surprise_criteria)
fear = Emotion('fear', [[1,2,4,5,7,20,26]], fear_criteria)
angry = Emotion('angry', [[4,5,7,23]], angry_criteria)
disgust = Emotion('disgust', [[9,15,16], [10,15,16]], disgus_criteria)
contempt = Emotion('contempt', [[12,14]], contempt_criteria)

emotions = [happy, sadness, surprise, fear, angry, disgust, contempt]



In [326]:
#emotion task - build training data
result = []
x_training = []
y_label = []

emotions = []
for data in svm_training_data:
    if(data.emotion != -1):
        tmp = []
        for vector in data.landmark:
            tmp.append(vector[0])
            tmp.append(vector[1])
        x_training.append(tmp)
        y_label.append(data.emotion)
        if(data.emotion not in emotions):
            emotions.append(data.emotion)
emotions.sort()
# print(x_training[1])


In [332]:
#emotion task - create training data for multi one-class SVM
x_training = []
for data in svm_training_data:
        if(data.emotion != -1):
            tmp = []
            for vector in data.landmark:
                tmp.append(vector[0])
                tmp.append(vector[1])
            x_training.append(tmp)
            
emotion_models = []
for e in emotions:
    clf = svm.LinearSVC()
    y_label = []
    for data in svm_training_data:
        if(data.emotion != -1):
            if(data.emotion == e):
                y_label.append(1)
            else:
                y_label.append(0)
    clf.fit(normalize(x_training, norm='max', axis=0), y_label)
    emotion_models.append(clf)
    print([e, clf.score(normalize(x_training, norm='max', axis=0), y_label)])
            

[1, 1.0]
[2, 0.99250936329588013]
[3, 1.0]
[4, 1.0]
[5, 1.0]
[6, 1.0]
[7, 0.99625468164794007]


In [328]:
#emotion task - evaluate multi one-class SVM
x_training = []
emotion_gt = []
for data in svm_testing_data:
        if(data.emotion != -1):
            tmp = []
            for vector in data.landmark:
                tmp.append(vector[0])
                tmp.append(vector[1])
            x_training.append(tmp)
            emotion_gt.append(data.emotion)
            
for e in emotions:
    y_label = []
    for data in svm_testing_data:
        if(data.emotion != -1):
            if(data.emotion == e):
                y_label.append(1)
            else:
                y_label.append(0)
    print([e, emotion_models[emotions.index(e)].score(normalize(x_training, norm='max', axis=0), y_label)])
    
    

[1, 0.85106382978723405]
[2, 0.61702127659574468]
[3, 1.0]
[4, 0.14893617021276595]
[5, 0.91489361702127658]
[6, 0.82978723404255317]
[7, 0.91489361702127658]


In [329]:
x_normalize = normalize(x_training, norm='max', axis=0)
for i in range(0,len(x_normalize)):
    predict = []
    print("---------------")
    print("ground true:" + str(emotion_gt[i]))
    for e in emotions:
        if(emotion_models[emotions.index(e)].predict([x]) == 1):
            predict.append(e)
    print("predict:")
    print(predict)
    print("---------------")
    

---------------
ground true:7
predict:
[4]
---------------
---------------
ground true:5
predict:
[4]
---------------
---------------
ground true:7
predict:
[4]
---------------
---------------
ground true:6
predict:
[4]
---------------
---------------
ground true:1
predict:
[4]
---------------
---------------
ground true:5
predict:
[4]
---------------
---------------
ground true:7
predict:
[4]
---------------
---------------
ground true:6
predict:
[4]
---------------
---------------
ground true:5
predict:
[4]
---------------
---------------
ground true:4
predict:
[4]
---------------
---------------
ground true:7
predict:
[4]
---------------
---------------
ground true:5
predict:
[4]
---------------
---------------
ground true:6
predict:
[4]
---------------
---------------
ground true:2
predict:
[4]
---------------
---------------
ground true:2
predict:
[4]
---------------
---------------
ground true:2
predict:
[4]
---------------
---------------
ground true:2
predict:
[4]
-------------

In [314]:
i

47

In [296]:
#emotion task - multiclass SVM
clf = svm.LinearSVC()
clf.fit(normalize(x_training, norm='max', axis=0), y_label)
print(clf.score(normalize(x_training, norm='max', axis=0), y_label))

x_training = []
y_label = []
for data in svm_testing_data:
    if(data.emotion != -1):
        tmp = []
        for vector in data.landmark:
            tmp.append(vector[0])
            tmp.append(vector[1])
        x_training.append(tmp)
        y_label.append(data.emotion)

print(clf.predict(normalize(x_training, norm='max', axis=0)))
print(y_label)
print(clf.score(normalize(x_training, norm='max', axis=0), y_label))

0.574468085106
[2 5 7 2 6 5 7 2 2 4 2 5 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 1 2 2 1 2 2 1 2 2
 2 2 2 2 2 4 2 2 2 2]
[7, 5, 7, 6, 1, 5, 7, 6, 5, 4, 7, 5, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 4, 6, 1, 2, 4, 1, 2, 6, 1, 2, 4, 6, 2, 6, 1, 2, 4, 6, 2, 1, 4]
0.574468085106


In [6]:
#save model
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
        
save_object(models, '/Programing/GR/Code/Python/models/au_models.pkl')
#save_object(facs, '/Programing/GR/Code/Python/models/facs.pkl')

In [23]:
#create json log file
import json
log_path = '/Programing/GR/Code/Python/log/'
import time
import json
ts = int(time.time())
#jsonAdder = json.dumps()
output = {'n_train': len(training_images), 'n_test': len(testing_images), 'au_score' : au_models_score, 'facs': facs }
#print json.dumps(output, sort_keys=True, indent=4, separators=(',', ': '))
with open(log_path + 'system_log' + str(ts) + '.txt', "w+") as outfile:
    json.dump(output, outfile,sort_keys=True, indent=4, separators=(',', ': '))